In [16]:
from bs4 import BeautifulSoup
import requests
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

base_url = 'https://bachtrack.com/find-opera/category=2;datefrom=20180501;medium=1'

In [ ]:
productions = []

for page in range(25):
    print(page)
    url = base_url + ';startrow=' + str(page*50)    
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    listings = soup.find_all(attrs={'class': 'listing-shortform'})
    for listing in listings:
        listing_url = 'https://bachtrack.com' + listing.find(attrs={'class': 'listing-more-info'}).get('href')
        listing_deets = BeautifulSoup(requests.get(listing_url, headers=headers).text, "html5lib")
        venue = listing_deets.find(attrs={'class': 'listing-venue'}).text
        dates = listing_deets.find(attrs={'class': 'listing-main-date'}).text
        city = listing_deets.find(attrs={'class': 'addressLocality'}).get_text() 
        country = listing_deets.find(itemprop="addressCountry").get_text()                 
        programme = listing_deets.find(attrs={'id': 'tbody_listing-programme'})
        if programme == None:
            print(listing_url)
        else:    
            for row in programme.find_all('tr'):
                detail = []
                for item in row.find_all('a'):
                    if '/composer/' in item.get('href'):
                        composer = item.get_text()
                for item in row.find_all('a'):
                    if '/work/' in item.get('href'):
                        work = item.get_text()
                        detail.append(country)
                        detail.append(city)
                        detail.append(venue)                    
                        detail.append(dates)
                        detail.append(composer)
                        detail.append(work)
                        detail.append(listing_url)                    
                        productions.append(detail)

In [226]:
import re

years = ['2017', '2018', '2019']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
        '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
        '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 
        '31', '1', '2', '3', '4', '5', '6', '7', '8', '9']    
dow = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def get_dates(date_str):

    dates_list = []        
    dates = re.split('; |, | |\*|\n',date_str)
    year = str(2018)
    month = ''
    day = ''
    
    if any(ext in dates for ext in dow):
        for idx in range(len(dates)):
            if dates[idx] in days:
                day = dates[idx]
            if dates[idx] in months:
                month = dates[idx]
            if dates[idx] in years:
                year = dates[idx]
                new_date = year + '-' + month + '-' + day
                dates_list.append(new_date)
    else:            
        for idx in range(len(dates)):
            if dates[idx] in years:
                year = dates[idx]
            if dates[idx] in months:
                month = dates[idx]
            if dates[idx] in days:
                new_date = year + '-' + month + '-' + str(dates[idx])
                dates_list.append(new_date)
    return dates_list

In [228]:
listings = []
for production in productions:
    dates = get_dates(production[3])
    for date in dates:        
        listing = []
        listing.append(production[0])
        listing.append(production[1])
        listing.append(production[2])
        listing.append(date)
        listing.append(production[4])
        listing.append(production[5])
        listing.append(production[6])
        listings.append(listing)

In [231]:
import pandas as pd                     
headers = ['country', 'city', 'venue', 'date', 'composer', 'work', 'listing_url']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../data/raw/bachtrack_listings.csv')